In [1]:
import requests
import html
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup as bs

import time
import os
import seaborn as sns
import pandas as pd
import scipy as sc
import numpy as np
import re

import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 

# Data Acquisition & Data Cleaning & Data Vectorization

# Part 1 - Load and clean the data
The following cells perform 2 things:
* load the csv file which contain the dataframe of spotify songs
* replace all of the unnecessary punctuation in each title and artist for the further work with scraping

In [2]:
df_spotify=pd.read_csv("./data/spotify.csv")
df_spotify=df_spotify.drop('Unnamed: 0', 1)    

## Define a function to clean the titles and artists string for the search in metrolyrics

In [3]:
def cleanDf(df):
    for i in range(0,len(df)):
        df['Title'][i]=df['Title'][i].replace('MotorSport','motor sport').replace('PILLOWTALK','pillow talk').replace('Back To You','Back 2 you')
        
        list=df['Title'][i].split(' (feat. ')
        df['Title'][i]=list[0]
        list=df['Title'][i].split('- From ')
        df['Title'][i]=list[0]
        list=df['Title'][i].split(' feat. ')
        df['Title'][i]=list[0]
        list=df['Title'][i].split(' (with ')
        df['Title'][i]=list[0]
        list=df['Title'][i].split(' (Original ')
        df['Title'][i]=list[0]
        list=df['Title'][i].split(' (From ')
        df['Title'][i]=list[0]
        list=df['Title'][i].split(' (Fifty ')
        df['Title'][i]=list[0]
        
        df['Title'][i]=df['Title'][i].replace("Wanna","want to").replace('\n','').replace(';','').replace("'","").replace(',','').replace('/ ','').replace('- ','').replace('-','').replace('é','e').replace('?','').replace('"','').replace("!","").replace("in'","ing").replace(" and "," ").replace("’","")
        df['Title'][i]=df['Title'][i].replace('(','').replace(')','').replace('.','')
        
        df['Artist'][i]=df['Artist'][i].replace('P!nk','Pink').replace('NERD','nerd the neptunes').replace('ZAYN','zayn malik').replace('Axwell /\ Ingrosso','Axwell Ingrosso').replace('Ayo & Teo','Ayo Teo')
        df['Artist'][i]=df['Artist'][i].replace('é','e').replace('í','i').replace('.','').replace('\n','').replace(' + ','-').replace('.','').replace('ó','o').replace('$','s').replace("'","").replace("!","")

## Clean our dataframe

In [4]:
cleanDf(df_spotify)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

## In the next cell we create a function to add a column to our dataframe

In [5]:
def append_col(df,lst,colName):
    df[colName] = lst

## Checking the amount of rows in dataframe

In [6]:
len(df_spotify)

587

# Part 2 - Load and store lyrics data


## Define a function to create the lyrics_urls list to store each song's lyrics url on metrolyrics
The following function perform 5 things:
* create list of urls from metrolyrics for each song 
* replace all of the unnecessary strings from the title of each song
* handle with cases of several artists of a song. split them and take the first one to build with it a url
* replace all of the spaces to '-' for the url
* clean the text from unnecessary punctuation

In [7]:

def create_lyrics_urls(df):
    #creating the list of urls for each song's lyrics
    lyrics_urls=[]
    for i in range (0,len(df)):
        #the case of & in songs:
        name=df['Title'][i].replace("& ","").replace("'","")
        name=name.replace(" ","-")

        list=df['Artist'][i].split(',')
        #we will always take the first artist
        artist_str=list[0]
        list=artist_str.split(' & ')
        artist_str=list[0]
        list=artist_str.split(' and ')
        artist_str=list[0]
        list=artist_str.split('featuring')
        artist_str=list[0]

        url='https://www.metrolyrics.com/'+name+'-lyrics-'+artist_str.replace('"','').replace(" ","-")+'.html'
        #the case of "Maroon-5-.html"
        url=url.replace("-.",".") 

        lyrics_urls.append(url)      
    return lyrics_urls    

In [8]:
urls=create_lyrics_urls(df_spotify)
len(urls) 

587

### Loading the nltk package for the further work with text

In [9]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords, movie_reviews

## Define function to delete the stopwords from the lyrics

In [10]:
stop_words = set(stopwords.words('english'))

def remove_mystopwords(string):
    tokens = string.split(" ")
    tokens_filtered = [word for word in tokens if not word in stop_words]
    return (" ").join(tokens_filtered)

## Define function to count the stopwords in each song

In [11]:
def count_stopwords(string):
    count=0
    tokens = string.split(" ")
    for word in tokens:
        if word in stop_words:
            count+=1
    return count
    

## The following cell pefrom 5 things:

* scrap the metrolyrics
* load the lyrics of each song
* check the amount of words in each song 
* check the amount of stopwords in each song 
* clean the text from digits, stopwords, and unnecessary punctuation


In [12]:
#the part of creation of the lyrics_texts dictionary which contain the lyrics of each song in top 2016,2017,2018

all_words_count=[]
stopwords_count=[]

def find_lyrics_txt(df):
    lyrics_texts=[]
    time.sleep(7)
    for i in range (0,len(df)):
        url = urls[i]
        response = requests.get(url)
        data = response.text
        soup = bs(data,'html.parser')
        couplets = soup.findAll("p",{"class":"verse"})
        temp = ""
        for p in couplets:
            temp = str(temp) + str(p.text)
            temp=temp+"\n"
        #cleaning the text from unnecessary punctuation
        temp=re.sub(r'[^(a-zA-Z)\s]','', temp)
        temp=temp.replace('[','').replace(']','').replace('(','').replace(')','').replace('"','').replace("'","")
        #remove all of the digits 
        temp=''.join(j for j in temp if not j.isdigit())
        #count the number of words in each song
        all_words_count.append(len(temp))
        #count the number of stopwords in each song
        stopwords_c=count_stopwords(temp)
        stopwords_count.append(stopwords_c)
        #remove all of the stopwords
        temp = remove_mystopwords(temp)
        #for the further work with text we would like to lowercase all of the words in text   
        lyrics_texts.append(temp.lower())
        lyrics_texts[i]=lyrics_texts[i].replace('\n ','\n')  
    return lyrics_texts    

In [13]:
lyrics=find_lyrics_txt(df_spotify)

## Add 2 columns to our dataframe

In [14]:
append_col(df_spotify,all_words_count,'words_count')
append_col(df_spotify,stopwords_count,'stopwords_count')
print(stopwords_count)

[160, 122, 129, 129, 184, 157, 116, 116, 170, 96, 67, 142, 113, 133, 122, 113, 46, 126, 231, 137, 131, 115, 124, 155, 77, 149, 69, 126, 169, 204, 171, 175, 162, 189, 134, 71, 152, 170, 142, 156, 108, 71, 83, 102, 229, 155, 94, 128, 72, 135, 95, 220, 160, 56, 130, 99, 226, 149, 148, 104, 114, 91, 104, 102, 122, 105, 30, 224, 190, 172, 111, 102, 248, 176, 154, 173, 137, 71, 189, 145, 147, 114, 99, 193, 45, 83, 125, 195, 216, 183, 168, 103, 68, 79, 91, 216, 167, 134, 249, 70, 191, 201, 60, 387, 96, 105, 167, 126, 105, 274, 187, 92, 184, 123, 129, 119, 168, 149, 107, 201, 0, 187, 74, 0, 74, 193, 92, 160, 161, 84, 105, 163, 190, 252, 184, 0, 216, 134, 221, 180, 173, 164, 132, 94, 153, 211, 170, 14, 66, 161, 377, 123, 206, 162, 131, 155, 117, 248, 195, 113, 108, 84, 0, 104, 163, 164, 189, 176, 123, 215, 153, 62, 100, 138, 70, 137, 161, 189, 103, 137, 135, 96, 245, 116, 154, 100, 182, 79, 126, 291, 133, 193, 127, 155, 122, 93, 110, 103, 115, 146, 127, 162, 166, 135, 148, 171, 139, 238, 201, 1

In [15]:
print(all_words_count)

[2057, 1479, 1773, 2704, 2494, 3278, 1592, 1568, 2538, 1770, 1572, 2059, 1355, 1664, 1950, 1994, 1207, 2012, 3150, 1987, 1686, 1622, 1832, 2112, 1216, 1842, 1011, 1362, 2459, 4137, 1933, 2061, 2178, 2712, 1762, 1980, 2572, 2569, 2143, 1667, 1965, 1461, 1077, 1356, 2634, 2145, 1490, 2669, 1300, 2069, 2183, 2788, 1880, 2100, 1811, 1449, 3696, 2197, 1991, 1734, 1607, 1370, 1551, 1416, 2015, 1708, 807, 2949, 1771, 2208, 1385, 1548, 2692, 2367, 2098, 2339, 2534, 945, 2352, 2000, 1795, 1597, 1208, 3024, 528, 1834, 1913, 2772, 2439, 3158, 1996, 1572, 1130, 1415, 1866, 3500, 1876, 2171, 2958, 3072, 2318, 2737, 1622, 5653, 1770, 1324, 1970, 2344, 1628, 4046, 2240, 1865, 2325, 2171, 1372, 2020, 2858, 2197, 1519, 2360, 0, 3034, 1229, 0, 1497, 3133, 1542, 2266, 2441, 1289, 1602, 2371, 3265, 3228, 2097, 0, 2300, 1675, 2669, 1944, 2339, 3084, 1502, 1490, 2279, 3223, 2538, 1187, 1494, 2859, 3819, 1587, 2841, 2363, 1686, 1932, 2315, 2833, 3436, 1650, 1742, 1166, 0, 1372, 2669, 2639, 2712, 2425, 1690, 

In [16]:
lyrics_without_empty_strings = []
for string in lyrics:
    if (string != ""):
        lyrics_without_empty_strings.append(string)

## In the next cell we want to check the amount of songs with zero words (the algorithm of finding the url for metrolyrics didn't work for them) so we can delete them in the dataframe in the future

In [17]:
res_list = [] 
count=0
for i in range(0, len(all_words_count)) : 
    if all_words_count[i] == 0 : 
        res_list.append(i)    
print(res_list)        

[120, 123, 135, 162, 241, 246, 279, 298, 300, 301, 302, 308, 311, 315, 316, 317, 318, 319, 320, 329, 333, 338, 341, 353, 363, 376, 377, 378, 383, 385, 389, 392, 399, 404, 406, 410, 414, 415, 417, 430, 432, 435, 441, 444, 445, 451, 453, 455, 458, 470, 472, 477, 480, 482, 483, 485, 486, 487, 499, 503, 511, 521, 522, 524, 531, 533, 538, 541, 545, 557, 562, 565, 569, 580]


## Cleaning: In the next cell we drop every row which value of amout of words in song is 0

In [18]:

for i in range (0,len(df_spotify)):
    if (df_spotify['words_count'][i]) == 0:
        df_spotify['words_count'][i]=pd.np.NaN
        
df_spotify=df_spotify.dropna()

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


## Check the amount of rows in a new dataframe without nans

In [19]:
len(df_spotify)

513

## Defining a text as positive, negative or neutral using textblob package

In [20]:
# !pip install textblob

## Getting the polarity value for each song and defining the range for each pnn value: 

* pnn between -0.3 and 0.3 is neutral and its value will marked as 0
* pnn above 0.3 is positive and its value will marked as 1
* pnn below -0.3 is negative and its value will marked as -1

In [21]:
from textblob import TextBlob
# The sentiment function of textblob returns two properties, polarity, and subjectivity.
# Polarity is float which lies in the range of [-1,1] where 1 means positive statement and -1 means a negative statement

def pos_neg_neutral(lyrics_texts):
    blobs=[]
    for i in range (0,len(lyrics_texts)):
        blob=TextBlob(lyrics_texts[i])
        pnn=blob.sentiment.polarity
        definition=0
        if -0.3 < pnn < 0.3:
            definition=0
        if pnn  <= -0.3:
            definition=-1
        if pnn >= 0.3:
            definition=1
        blobs.append(definition)    
    return blobs

In [22]:
blobs_polarity=pos_neg_neutral(lyrics_without_empty_strings)

In [23]:
len(blobs_polarity)

513

In [24]:
print(blobs_polarity)

[0, -1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0

## In the next cell we will check hom many negative,neutral,positive songs do we have

In [25]:
blobs_res={"pos": 0, "neg": 0, "neutral": 0}
for i in range (0,len(blobs_polarity)):
    if blobs_polarity[i]==1:
        blobs_res["pos"]+=1
    if blobs_polarity[i]==-1:
        blobs_res["neg"]+=1
    if blobs_polarity[i]==0:
        blobs_res["neutral"]+=1
print(blobs_res)        

{'pos': 50, 'neg': 7, 'neutral': 456}


In [26]:
append_col(df_spotify,blobs_polarity,'blobs_polarity')

## In the next cell we will tokenize (convert the whole lyrics text to words) the lyrics of each song

In [27]:
tokenized_lyrics=[]

for i in range (0,len(df_spotify)):
    tokens=word_tokenize(lyrics_without_empty_strings[i])
    tokenized_lyrics.append(tokens)

In [28]:
print(str(tokenized_lyrics[0]))

['for', 'times', 'rain', 'parade', 'and', 'clubs', 'get', 'using', 'name', 'you', 'think', 'broke', 'heart', 'oh', 'girl', 'goodness', 'sake', 'you', 'think', 'im', 'crying', 'well', 'i', 'aint', 'and', 'i', 'didnt', 'wan', 'na', 'write', 'song', 'cause', 'i', 'didnt', 'want', 'anyone', 'thinking', 'i', 'still', 'care', 'i', 'dont', 'still', 'hit', 'phone', 'up', 'and', 'baby', 'i', 'movin', 'on', 'and', 'i', 'think', 'somethin', 'i', 'dont', 'wan', 'na', 'hold', 'back', 'maybe', 'know', 'that', 'my', 'mama', 'dont', 'like', 'likes', 'everyone', 'and', 'i', 'never', 'like', 'admit', 'i', 'wrong', 'and', 'ive', 'caught', 'job', 'didnt', 'see', 'whats', 'going', 'on', 'but', 'i', 'know', 'im', 'better', 'sleeping', 'own', 'cause', 'like', 'way', 'look', 'much', 'oh', 'baby', 'go', 'love', 'yourself', 'and', 'think', 'im', 'still', 'holdin', 'somethin', 'you', 'go', 'love', 'yourself', 'and', 'told', 'hated', 'friends', 'the', 'problem', 'them', 'and', 'every', 'time', 'told', 'opinion', 

## The next 3 cells perfom 3 things:

* create a list with positive words by scraping a web page that includes the necessary information for this and count the total amount of these words in each song
* create a list with negative words by scraping a web page that includes the necessary information for this and count the total amount of these words in each song
* create a list with curse words by scraping a web page that includes the necessary information for this and count the total amount of these words in each song

In [29]:
count_pos=[]

keywords_pos=[]

url='https://www.enchantedlearning.com/wordlist/positivewords.shtml'
response = requests.get(url)
data = response.text
soup = bs(data,'html.parser')
words = soup.findAll("div",{"class":"wordlist-section"})

for div in words:
    divs=soup.findAll("div",{"class":"wordlist-item"})
    for div in divs:
        keywords_pos.append(div.text)

for i in range (0,len(df_spotify)):
    temp=[]
    for word in keywords_pos:
        #here we create a temp list to store all the words from keywords_neg which appear in lyrics 
        temp+=re.findall(word, str(lyrics_without_empty_strings[i]))
    #now we'd want to know how many times these words appeared in text
    count_pos.append(len(temp))
print(count_pos)         

[350, 600, 525, 450, 525, 200, 200, 400, 850, 425, 375, 300, 275, 825, 675, 175, 150, 450, 800, 200, 400, 875, 150, 900, 225, 950, 300, 325, 575, 550, 275, 175, 675, 350, 450, 200, 300, 350, 375, 425, 200, 425, 775, 125, 325, 275, 175, 325, 300, 575, 400, 1050, 275, 625, 200, 175, 1875, 450, 1025, 325, 275, 75, 250, 450, 275, 350, 100, 400, 225, 725, 325, 250, 1050, 525, 475, 275, 1000, 175, 525, 375, 175, 950, 150, 2375, 175, 1250, 625, 375, 275, 425, 125, 675, 100, 500, 275, 525, 200, 800, 125, 50, 675, 475, 550, 700, 425, 275, 500, 825, 375, 2450, 325, 200, 650, 600, 200, 275, 475, 450, 25, 425, 725, 375, 200, 400, 325, 650, 575, 350, 125, 575, 725, 925, 425, 225, 275, 250, 250, 275, 600, 450, 175, 575, 700, 850, 0, 400, 225, 1275, 450, 500, 250, 400, 1325, 625, 450, 475, 500, 625, 175, 300, 300, 275, 350, 375, 375, 650, 675, 225, 350, 1250, 175, 0, 525, 225, 675, 475, 400, 275, 875, 450, 175, 125, 175, 300, 325, 250, 325, 425, 125, 400, 350, 0, 175, 175, 525, 400, 125, 250, 250, 27

In [30]:
count_neg=[]

keywords_neg=[]

url='https://www.enchantedlearning.com/wordlist/negativewords.shtml'
response = requests.get(url)
data = response.text
soup = bs(data,'html.parser')
words = soup.findAll("div",{"class":"wordlist-section"})

for div in words:
    divs=soup.findAll("div",{"class":"wordlist-item"})
    for div in divs:
        keywords_neg.append(div.text)
        
for i in range (0,len(df_spotify)):
    temp=[]
    for word in keywords_neg:
        #here we create a temp list to store all the words from keywords_neg which appear in lyrics 
        temp+=re.findall(word, str(lyrics_without_empty_strings[i]))
    #now we'd want to know how many times these words appeared in text
    count_neg.append(len(temp))
print(count_neg)  

[648, 1104, 384, 816, 336, 600, 312, 288, 480, 840, 552, 1032, 504, 456, 1008, 240, 72, 624, 1032, 336, 264, 672, 672, 600, 600, 360, 456, 360, 288, 552, 960, 168, 336, 312, 456, 288, 456, 1272, 456, 1320, 408, 672, 576, 336, 960, 600, 624, 672, 240, 456, 216, 1656, 360, 192, 216, 288, 312, 360, 672, 552, 408, 192, 168, 408, 360, 1104, 576, 1056, 240, 312, 504, 168, 960, 504, 456, 480, 1056, 96, 600, 432, 456, 120, 168, 2976, 168, 1008, 840, 648, 216, 336, 168, 528, 72, 168, 2064, 1080, 432, 432, 192, 360, 288, 408, 120, 1968, 840, 240, 456, 792, 552, 840, 672, 408, 1152, 288, 144, 432, 1032, 360, 408, 336, 672, 264, 312, 816, 360, 600, 624, 480, 288, 288, 624, 576, 144, 912, 312, 768, 96, 480, 1152, 720, 624, 1488, 192, 480, 192, 120, 576, 504, 216, 384, 312, 264, 1296, 888, 336, 576, 456, 552, 432, 312, 168, 1728, 312, 216, 240, 384, 864, 288, 144, 1008, 144, 360, 648, 168, 528, 288, 288, 480, 624, 576, 168, 240, 96, 168, 360, 264, 384, 672, 96, 528, 984, 0, 1608, 504, 336, 624, 96, 

In [31]:
count_swear=[]

keywords_swear=[]

url='https://en.wiktionary.org/wiki/Category:English_swear_words'
response = requests.get(url)
data = response.text
soup = bs(data,'html.parser')
div_class = soup.findAll("div",{"class":"mw-category-group"})

for div in div_class:
    uls=div.find_all('ul')
    for ul in uls:
        lis=ul.find_all('li')
        for ls in lis:
            links=ls.find_all('a')
            for link in links:
                keywords_swear.append(link.text)
                
keywords_swear.pop(0)         

for k in range(0,len(keywords_swear)):
    keywords_swear[k] =keywords_swear[k].lower() 

# print(keywords_swear)

for i in range (0,len(df_spotify)):
    temp=[]
    for word in keywords_swear:
        #here we create a temp list to store all the words from keywords_neg which appear in lyrics 
        temp+=re.findall(word, str(lyrics_without_empty_strings[i]))
    #now we'd want to know how many times these words appeared in text
    count_swear.append(len(temp))
print(count_swear)  

[0, 0, 0, 1, 0, 24, 9, 0, 0, 0, 0, 0, 14, 2, 1, 0, 0, 8, 8, 0, 0, 3, 0, 2, 0, 0, 0, 0, 0, 28, 0, 17, 0, 25, 6, 0, 4, 10, 0, 3, 0, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 5, 12, 1, 0, 11, 13, 6, 16, 6, 3, 0, 0, 11, 6, 0, 1, 5, 1, 4, 0, 7, 11, 0, 2, 0, 1, 0, 10, 0, 0, 0, 0, 1, 0, 10, 10, 16, 6, 2, 3, 0, 0, 11, 41, 1, 0, 0, 1, 1, 51, 0, 50, 0, 1, 1, 8, 0, 5, 2, 19, 7, 4, 1, 3, 6, 13, 0, 0, 22, 8, 0, 8, 1, 0, 1, 0, 3, 0, 23, 0, 1, 0, 1, 1, 1, 2, 0, 0, 2, 7, 25, 0, 0, 2, 23, 70, 0, 10, 13, 0, 0, 21, 1, 13, 0, 16, 20, 2, 4, 1, 25, 4, 1, 3, 3, 6, 0, 0, 0, 20, 9, 23, 3, 7, 0, 4, 21, 0, 0, 0, 0, 0, 0, 18, 2, 27, 1, 0, 16, 25, 0, 0, 3, 0, 1, 13, 7, 8, 0, 16, 1, 10, 2, 1, 3, 0, 1, 1, 0, 3, 0, 2, 2, 28, 2, 6, 0, 1, 23, 1, 44, 15, 0, 0, 14, 3, 1, 0, 20, 3, 4, 0, 17, 7, 27, 0, 4, 3, 0, 1, 24, 0, 22, 2, 46, 21, 0, 16, 0, 44, 0, 21, 7, 0, 0, 0, 1, 1, 0, 0, 18, 4, 15, 4, 0, 0, 0, 20, 0, 1, 1, 1, 0, 6, 0, 18, 0, 0, 5, 0, 0, 5, 1, 0, 30, 9, 0, 1, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 0, 0, 1, 11, 0, 8, 2, 1, 0, 6, 0, 0

In [32]:
append_col(df_spotify,count_neg,'words_neg_count')
append_col(df_spotify,count_pos,'words_pos_count')
append_col(df_spotify,count_swear,'words_curse_count')

## In the next cell we'd like to find the total count of repeated words and the most common words in each song using Counter

In [33]:
from collections import Counter

reg = re.compile('\S{4,}')

repeated_words=[]
most_common_words=[]

for i in range (0,len(df_spotify)):
    c = Counter(ma.group() for ma in reg.finditer(lyrics_without_empty_strings[i]))
    repeated_words.append(sum(c.values()))
    most_common_words.append([k for k,v in c.most_common()])

In [34]:
type(most_common_words[0])

list

In [35]:
avg_most_common_words_size=[]

for i in range(0,len(most_common_words)):
    total=0
    for word in most_common_words[i]:
        total += len(word)
    ave_size = round(float(total) / float(len(most_common_words[i])) , 2) 
    avg_most_common_words_size.append(ave_size)
        
print(avg_most_common_words_size)        

[5.08, 5.34, 4.98, 5.36, 5.29, 5.42, 5.51, 5.27, 5.42, 5.03, 4.86, 5.54, 5.19, 5.09, 5.34, 5.28, 5.5, 5.64, 5.4, 5.41, 5.37, 5.41, 5.19, 5.41, 5.41, 5.19, 5.14, 5.31, 5.53, 5.53, 5.49, 4.97, 5.36, 5.36, 5.4, 5.82, 5.66, 5.27, 5.71, 5.11, 5.12, 5.49, 4.83, 5.52, 5.38, 5.25, 5.5, 5.23, 5.36, 4.95, 5.15, 5.3, 5.49, 5.68, 5.07, 5.1, 5.65, 5.15, 5.2, 5.38, 5.63, 4.96, 5.35, 5.41, 5.34, 5.33, 4.93, 5.55, 5.22, 4.92, 4.9, 5.13, 4.83, 5.56, 5.54, 5.32, 5.01, 5.41, 5.25, 5.61, 5.19, 5.34, 5.05, 4.97, 5.4, 4.94, 5.01, 5.31, 5.27, 6.06, 5.52, 5.5, 5.2, 5.24, 5.87, 5.32, 5.38, 5.44, 5.18, 5.98, 5.34, 5.45, 5.8, 5.25, 5.03, 4.96, 5.22, 5.96, 5.01, 5.34, 5.35, 5.95, 4.87, 5.17, 5.15, 5.21, 5.31, 5.15, 5.54, 5.27, 4.98, 5.07, 5.6, 5.34, 4.88, 5.85, 5.26, 5.12, 5.26, 4.78, 5.39, 4.97, 4.93, 5.11, 5.54, 5.31, 5.71, 5.32, 5.67, 4.97, 5.5, 5.28, 5.26, 5.42, 5.74, 5.67, 5.29, 5.22, 5.18, 5.36, 5.39, 5.37, 4.91, 5.14, 5.3, 6.27, 5.26, 5.11, 5.63, 5.43, 5.31, 5.37, 5.36, 5.63, 4.68, 5.39, 5.58, 5.67, 5.2, 5

In [36]:
append_col(df_spotify,avg_most_common_words_size,'avg_most_common_words_size')
append_col(df_spotify,repeated_words,'words_repeat_count')

In [37]:
df_spotify.to_csv('./data/spotify_after.csv',index=False)

## From this point we saw that we schould add an additional data to our dataframe 

In [38]:
df_spotify_add_pct = pd.read_csv('./data/spotify_after.csv')

In [39]:
df_spotify_add_pct.columns

Index(['Title', 'Artist', 'id', 'artist_genres', 'is_single', 'total_tracks',
       'release_date', 'day_of_week', 'release_season', 'Year', 'is_top100',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'words_count',
       'stopwords_count', 'blobs_polarity', 'words_neg_count',
       'words_pos_count', 'words_curse_count', 'avg_most_common_words_size',
       'words_repeat_count'],
      dtype='object')

## In the next cell we add new columns to the dataframe to see the percentage of:

* amount of negative words in the song
* amount of positive words in the song
* amount of curse words in the song
* amount of repeated words in the song
* amount of stopwords in the song

## to the total number of words in a song

In [40]:

df_spotify_add_pct['words_negative_pct'] = round((df_spotify_add_pct['words_neg_count'] / df_spotify_add_pct['words_count'])*100,2)
df_spotify_add_pct['words_positive_pct'] = round((df_spotify_add_pct['words_pos_count'] / df_spotify_add_pct['words_count'])*100,2)
df_spotify_add_pct['words_curse_pct'] = round((df_spotify_add_pct['words_curse_count'] / df_spotify_add_pct['words_count'])*100,2)
df_spotify_add_pct['words_repeat_pct'] = round((df_spotify_add_pct['words_repeat_count'] / df_spotify_add_pct['words_count'])*100,2)
df_spotify_add_pct['stopwords_pct'] = round((df_spotify_add_pct['stopwords_count'] / df_spotify_add_pct['words_count'])*100,2)


In [41]:
df_spotify_add_pct[:10]

,Title,Artist,id,artist_genres,is_single,total_tracks,release_date,day_of_week,release_season,Year,...,words_neg_count,words_pos_count,words_curse_count,avg_most_common_words_size,words_repeat_count,words_negative_pct,words_positive_pct,words_curse_pct,words_repeat_pct,stopwords_pct
0,Love Yourself,Justin Bieber,3hB5DgAiMAQ4DzYbsMq1IT,"['canadian pop', 'pop', 'post-teen pop']",0,19,2015-11-13,5,3,2015,...,648,350,0,5.08,179,31.50,17.02,0.00,8.70,7.78
1,Sorry,Justin Bieber,69bp2EbF7Q2rqc5N3ylezZ,"['canadian pop', 'pop', 'post-teen pop']",0,19,2015-11-13,5,3,2015,...,1104,600,0,5.34,117,74.65,40.57,0.00,7.91,8.25
2,One Dance,Drake,1xznGGDReH1oQq0xzbwXa3,"['canadian hip hop', 'canadian pop', 'hip hop'...",0,20,2016-05-06,5,1,2016,...,384,525,0,4.98,184,21.66,29.61,0.00,10.38,7.28
3,Work,Rihanna,14WWzenpaEgQZlqPq2nk4v,"['barbadian pop', 'dance pop', 'pop', 'post-te...",0,16,2016-01-28,4,4,2016,...,816,450,1,5.36,307,30.18,16.64,0.04,11.35,4.77
4,Stressed Out,Twenty One Pilots,3CRDbSIZ4r5MsZ0YwxuEkn,"['modern rock', 'rock']",0,14,2015-05-15,5,1,2015,...,336,525,0,5.29,265,13.47,21.05,0.00,10.63,7.38
5,Panda,Desiigner,5OOkp4U9P9oL23maHFHL1h,"['pop rap', 'rap', 'southern hip hop', 'trap',...",0,14,2016-06-26,7,2,2016,...,600,200,24,5.42,362,18.30,6.10,0.73,11.04,4.79
6,Hello,Adele,0ENSn4fwAbCGeFGVUbXEU3,"['british soul', 'pop', 'uk pop']",1,1,2015-10-23,5,3,2015,...,312,200,9,5.51,142,19.60,12.56,0.57,8.92,7.29
7,Dont Let Me Down,The Chainsmokers,1i1fxkWeaMmKEB4T7zqbzK,"['electropop', 'pop', 'tropical house']",1,1,2016-02-05,5,4,2016,...,288,400,0,5.27,150,18.37,25.51,0.00,9.57,7.40
8,CANT STOP THE FEELING,Justin Timberlake,6JV2JOEocMgcZxYSZelKcc,"['dance pop', 'pop']",1,1,2016-05-06,5,1,2016,...,480,850,0,5.42,252,18.91,33.49,0.00,9.93,6.70
9,Closer,The Chainsmokers,7BKLCZ1jbUBVqRi2FVlTVw,"['electropop', 'pop', 'tropical house']",1,1,2016-07-29,5,2,2016,...,840,425,0,5.03,196,47.46,24.01,0.00,11.07,5.42


In [42]:
df_spotify_add_pct.to_csv('./data/spotify_after.csv',index=False)